In [1]:
import os
import pandas as pd
import numpy as np
import boto3

In [3]:
# Create an S3 client
s3 = boto3.client('s3')

# Specify the bucket name
bucket_name = 'open-logo-s3'

# Get the list of objects in the bucket
response = s3.list_objects_v2(Bucket=bucket_name)

# Create an empty list to store the file information
file_info = []

# Iterate through the objects and extract the relevant information
while response['IsTruncated']:
    for obj in response['Contents']:
        file_info.append({
            'Key': obj['Key'],
            'Size': obj['Size'],
            'LastModified': obj['LastModified']
        })
    token = response['NextContinuationToken']
    response = s3.list_objects_v2(Bucket=bucket_name, ContinuationToken=token)

for obj in response['Contents']:
    file_info.append({
        'Key': obj['Key'],
        'Size': obj['Size'],
        'LastModified': obj['LastModified']
    })

# Create a Pandas DataFrame from the file information
df_names = pd.DataFrame(file_info)
df_names

,Key,Size,LastModified
0,open_logo.csv,2633611,2024-06-13 14:03:24+00:00
1,open_logo_annotations/1008198576.xml,607,2024-06-13 13:41:58+00:00
2,open_logo_annotations/1016381746.xml,603,2024-06-13 13:41:59+00:00
3,open_logo_annotations/102448630.xml,610,2024-06-13 13:41:59+00:00
4,open_logo_annotations/1030265231.xml,606,2024-06-13 13:41:59+00:00
...,...,...,...
55003,open_logo_images/zara5.jpg,57705,2024-06-13 13:42:51+00:00
55004,open_logo_images/zara6.jpg,53682,2024-06-13 13:42:51+00:00
55005,open_logo_images/zara7.jpg,230092,2024-06-13 13:42:51+00:00
55006,open_logo_images/zara8.jpg,107913,2024-06-13 13:42:52+00:00


In [5]:
import io
obj = s3.get_object(Bucket='open-logo-s3', Key='open_logo.csv')
df = pd.read_csv(io.BytesIO(obj['Body'].read()), sep = '\t')

In [9]:
train_val_ratio = 0.8
train_num = int(len(df)*0.8//1) #Number of files to be used for training
train_indices = np.random.choice(range(0, train_num), size=train_num, replace=False) #Indeces of training files
df[df.index.isin(train_indices)] #Dataframe with files to be used for training

21666

In [23]:
p = 's3:'
train_img_path = os.path.join(p, 'open-logo-s3', 'YOLO', 'data', 'images','train')

's3:\\YOLO\\data\\images\\train'

In [21]:

os.makedirs(train_img_path)
val_img_path = os.path.join(p, 'YOLO', 'data', 'images','val')
os.makedirs(val_img_path)
train_lbl_path = os.path.join(p, 'YOLO', 'data', 'labels','train')
os.makedirs(train_lbl_path)
val_lbl_path = os.path.join(p, 'YOLO', 'data', 'labels','val')
os.makedirs(val_lbl_path)

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 's3:'

In [ ]:
for i in range(len(df[~df.index.isin(train_indices)])):
  img_curr_path = df[~df.index.isin(train_indices)]['image_path']
  img_curr_path = os.path.join(p, 'raw', 'images', val_file)
  img_new_path = os.path.join(val_img_path, val_file)
  shutil.move(img_curr_path, img_new_path)
  lbl_curr_path = df[~df.index.isin(train_indices)]['annotations_path']
  lbl_new_path = os.path.join(val_lbl_path, val_file[:-3] +'txt')
  shutil.move(lbl_curr_path, lbl_new_path)